In [15]:
import pcl.detection_builder
import torch
import torch.nn as nn

In [3]:
model = pcl.detection_builder.DetectionCL(_, dim=128, r=8)

In [8]:
print(model.queue.shape)
print(model.queue2.shape)   # 每个特征图的维度为[1024, 7, 7]

torch.Size([128, 8])
torch.Size([1024, 7, 7, 8])


In [10]:
im_q, im_k = torch.randn((4, 3, 224, 224)), torch.randn((4, 3, 224, 224))
print(im_q.shape, im_k.shape)

In [12]:
im_q = im_q.contiguous()
im_k = im_k.contiguous()
print(im_q.shape, im_k.shape)

torch.Size([4, 3, 224, 224]) torch.Size([4, 3, 224, 224])


In [13]:
# compute query features
q_b_list = model.get_encoderq_features(im_q)  # backbone features
q_global = model.encoder_q[1](q_b_list[-1])
q_dense = model.encoder_q[2](q_b_list)

q_global = nn.functional.normalize(q_global, dim=1)
q_dense = nn.functional.normalize(q_dense, dim=1)

In [22]:
with torch.no_grad():   # no gradient to keys
    # 获取encoder_k输出
    k_b_list = model.get_encoderk_features(im_k)
    # mlp输出
    k_global = model.encoder_k[1](k_b_list[-1])
    k_dense = model.encoder_k[2](k_b_list)

    k_global = nn.functional.normalize(k_global, dim=1)  # global
    k_dense = nn.functional.normalize(k_dense, dim=1)  # dense

In [23]:
print(q_global.shape, q_dense.shape)

torch.Size([4, 128]) torch.Size([4, 1024, 7, 7])


In [24]:
l_global_pos = torch.einsum('nc,nc->n', [q_global, k_global]).unsqueeze(-1)  # positive samples
l_global_neg = torch.einsum('nc,ck->nk', [q_global, model.queue.clone().detach()])  # negative samples

In [25]:
l_global_pos.shape

torch.Size([4, 1])

In [36]:
k_global.shape

torch.Size([4, 128])

In [26]:
torch.einsum('nc,nc->n', [q_global, k_global])

tensor([0.9822, 0.9829, 0.9840, 0.9832], grad_fn=<ViewBackward0>)

In [29]:
torch.einsum('nc,ck->nk', [q_global, model.queue.clone().detach()])

tensor([[-0.1596,  0.2016, -0.0703, -0.0441,  0.1324, -0.0512, -0.0601,  0.0347],
        [-0.1796,  0.1616, -0.0501, -0.0399,  0.1262, -0.0181, -0.0649,  0.0521],
        [-0.1513,  0.1531, -0.0615, -0.0428,  0.1357, -0.0398, -0.0685,  0.0369],
        [-0.1699,  0.1717, -0.0449, -0.0447,  0.1368, -0.0249, -0.0482,  0.0137]], grad_fn=<ViewBackward0>)

In [43]:
q_dense_flat = q_dense.view(q_dense.size(0), q_dense.size(1), -1).permute(0, 2, 1)  # (N, H*W, C)
k_dense_flat = k_dense.view(k_dense.size(0), k_dense.size(1), -1).permute(0, 2, 1)  # (N, H*W, C)

In [44]:
q_dense_flat.shape

torch.Size([4, 49, 1024])

In [48]:
l_dense_pos = torch.einsum('nqc,nqc->nqc', [q_dense_flat, k_dense_flat])
# q_dense: torch.Size([4, 1024, 7, 7])
# k_dense: torch.Size([4, 1024, 7, 7])

In [49]:
l_dense_pos.shape

torch.Size([4, 49, 1024])

In [50]:
l_dense_pos

tensor([[[0.0000e+00, 7.1813e-04, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [7.7576e-04, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 1.1323e-03],
         ...,
         [8.2557e-04, 6.5190e-04, 0.0000e+00,  ..., 0.0000e+00, 7.2938e-04, 0.0000e+00],
         [0.0000e+00, 1.5761e-02, 1.2826e-03,  ..., 1.3470e-03, 4.5429e-05, 0.0000e+00],
         [0.0000e+00, 1.0023e-03, 2.1754e-03,  ..., 0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 2.2619e-03, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 8.5241e-04, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.7064e-03, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 3.5650e-04,  ..., 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.8164e-03, 1.6932e-04, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 1.31

In [39]:
torch.mm(q_dense.detach(), k_dense.detach())

RuntimeError: self must be a matrix

In [42]:
a = torch.rand(2,3)
b = torch.rand(3,4)
# c = torch.einsum("ik,kj->ij", [a, b])
torch.mm(a, b)

tensor([[0.9947, 0.3119, 1.0214, 0.7836],
        [0.2939, 0.5344, 0.4799, 0.8897]])

In [38]:
q_dense

tensor([[[[3.3653e-02, 0.0000e+00, 1.4795e-02,  ..., 6.4652e-02, 7.3639e-02, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 2.5613e-02,  ..., 0.0000e+00, 0.0000e+00, 1.2883e-02],
          [0.0000e+00, 1.8223e-02, 3.8392e-03,  ..., 0.0000e+00, 0.0000e+00, 5.4715e-02],
          ...,
          [0.0000e+00, 1.0703e-02, 0.0000e+00,  ..., 5.3265e-02, 2.0434e-02, 4.0874e-02],
          [0.0000e+00, 9.9974e-02, 0.0000e+00,  ..., 1.4287e-02, 0.0000e+00, 7.6658e-02],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.8607e-02, 0.0000e+00, 0.0000e+00]],

         [[5.2093e-02, 4.6193e-02, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 2.2967e-02],
          [8.8649e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 1.9647e-02],
          [7.4484e-02, 0.0000e+00, 0.0000e+00,  ..., 9.7274e-03, 0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00, 2.8991e-03],
          [0.0000e+00, 1.2798e-02, 0.0000e+00,  ..., 0.0000e+00, 0.0